In [1]:
import keras_tuner as kt
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

a = tfds.load('german_credit_numeric')
df = tfds.as_dataframe(a['train'])
features = pd.DataFrame(df['features'].tolist()).values
targets = df['label'].values


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

X_train, X_test, y_train,y_test = train_test_split(features, targets)

scaler = StandardScaler().fit(X_train)
X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)
pd.DataFrame(X_train).describe()

2022-01-05 11:55:47.699945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-05 11:55:47.699961: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-01-05 11:55:48.940813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-05 11:55:48.940829: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-05 11:55:48.940839: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parkerburchett-desktop): /proc/driver/nvidia/version does not exist
2022-01-05 11:55:48.940986: I tensorflow/core/platform

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,...,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02,7.500000e+02
mean,9.710751e-17,6.602126e-17,-3.019807e-16,-1.206442e-16,-7.096175e-17,-1.746751e-17,-1.653492e-16,4.221068e-16,-1.684578e-16,2.874367e-16,...,-1.571336e-16,2.353673e-17,9.414691e-17,-3.449093e-17,-4.366877e-18,2.304823e-16,-1.731948e-16,2.617166e-16,9.414691e-17,-5.269859e-17
std,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,...,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00,1.000667e+00
min,-1.255674e+00,-1.389418e+00,-2.367519e+00,-1.083393e+00,-7.175368e-01,-1.948150e+00,-2.402562e+00,-1.656978e+00,-1.314940e+00,-1.453101e+00,...,-1.932470e-01,-5.598974e-01,-3.308587e-01,-3.219260e+00,-2.145346e-01,-4.557735e-01,-1.619626e+00,-1.568125e-01,-5.041633e-01,-1.306745e+00
25%,-1.255674e+00,-7.182017e-01,-5.140333e-01,-6.903760e-01,-7.175368e-01,-2.934360e-01,-9.892902e-01,-7.551394e-01,-1.314940e+00,-7.580038e-01,...,-1.932470e-01,-5.598974e-01,-3.308587e-01,3.106304e-01,-2.145346e-01,-4.557735e-01,-1.619626e+00,-1.568125e-01,-5.041633e-01,-1.306745e+00
50%,-4.575763e-01,-2.147893e-01,-5.140333e-01,-3.330878e-01,-7.175368e-01,-2.934360e-01,4.239815e-01,1.466991e-01,-3.527889e-01,-2.366807e-01,...,-1.932470e-01,-5.598974e-01,-3.308587e-01,3.106304e-01,-2.145346e-01,-4.557735e-01,6.174265e-01,-1.568125e-01,-5.041633e-01,7.652605e-01
75%,1.138620e+00,2.886231e-01,1.339452e+00,2.743020e-01,5.368982e-01,5.339211e-01,4.239815e-01,1.048538e+00,6.093626e-01,5.453040e-01,...,-1.932470e-01,-5.598974e-01,-3.308587e-01,3.106304e-01,-2.145346e-01,-4.557735e-01,6.174265e-01,-1.568125e-01,-5.041633e-01,7.652605e-01
max,1.138620e+00,4.315922e+00,1.339452e+00,4.526031e+00,1.791333e+00,1.361278e+00,1.837253e+00,1.048538e+00,1.571514e+00,3.412581e+00,...,5.174725e+00,1.786041e+00,3.022439e+00,3.106304e-01,4.661252e+00,2.194072e+00,6.174265e-01,6.377042e+00,1.983484e+00,7.652605e-01


In [2]:
from CustomLayers import FeatureReversalNoise
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(24)))
model.add(FeatureReversalNoise(input_vector_length=24, prob=.2))
model.add(tf.keras.layers.Dense(8))
model.add(tf.keras.layers.Dense(1, activation='tanh'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.01), loss='mse')
model.fit(x=X_train,y=y_train, verbose=0, epochs=10, validation_split=.2)
y_preds = model.predict(X_test)
mean_squared_error(y_test,y_preds)

0.1762640134788755

In [3]:
from tensorflow import keras
def model_builder_factory(n_features,
                          prob,
                          min_layers,
                          max_layers,
                          min_hidden_nodes,
                          max_hidden_nodes,
                          include_prob):
  """
  returns the model_builder function based on the hyper parameters passed 
  """
  def hypermodel(hp):
    glorrot_init = tf.keras.initializers.GlorotUniform() # A smarter way to set inital weights
    model = keras.Sequential()
    model.add(tf.keras.Input(shape=(n_features)))
    if include_prob:
      model.add(FeatureReversalNoise(input_vector_length=n_features,prob=prob))
    
    nodes_per_layer = []
    for i in range(max_layers):
      hp_units = hp.Int(f'Dense_layer{i}',
                      min_value=min_hidden_nodes,
                      max_value=max_hidden_nodes,
                      step=1)
      nodes_per_layer.append(hp_units)

    for i in range(hp.Int('n_layers', min_value=min_layers, max_value=max_layers, step=1)):
      model.add(keras.layers.Dense(units=nodes_per_layer[i],
                                   activation='relu',
                                   kernel_initializer=glorrot_init))
      
    model.add(keras.layers.Dense(1,activation='tanh'))
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3,])
    hp_beta_1 = hp.Choice('beta_1', values=[.8,.9,.92,.94,.95])
    hp_beta_2 = hp.Choice('beta_2', values=[.98,.99,.995,.999])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=.01,
                                                  beta_1=hp_beta_1,
                                                  beta_2=hp_beta_2),
                  loss='mse', metrics=['mse'])

    return model

  return hypermodel


hypermodel = model_builder_factory(24,.1,1,3,2,8,include_prob=True)
tuner = kt.RandomSearch(hypermodel=hypermodel,
                          objective='mse',
                          max_trials=10,
                          overwrite=True,
                          #directory='/kerasproject',
                          project_name='testLayers')
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=3)
tuner.search(x=X_train,y=y_train,      
            verbose=0,
            epochs=10,
            batch_size=2**8)

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [4]:
best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(X_train,y_train,epochs=20,batch_size=2**8,verbose=0)
y_preds_in_memory = best_model.predict(X_test)

best_model.save(f'model.h5',save_format='h5')
saved_model = tf.keras.models.load_model("model.h5")
y_preds_from_disk = saved_model.predict(X_test)
print('Are the predictions of the model in memory are the same as the model loaded from disk?')
np.all(y_preds_in_memory == y_preds_from_disk)

Are the predictions of the model in memory are the same as the model loaded from disk?


True

These values should be the same. If they are not that means you got an error